In [1]:
import pandas as pd
import cpca

In [2]:
read_csv_path = './recruitment-info.csv'
save_csv_path = './result1-1.csv'

frame_data = pd.read_csv(read_csv_path, encoding='GBK')

In [3]:
frame_data.head(3)

,job_id,company_full_name,company_short_name,position_name,job_min_wage,job_max_wage,job_wage_kind,require_kind,require_edu,require_exp,num_people,deadline,company_ind,position_keys,skill_keys,job_require,job_welfare,company_addr
0,1631112859985510400,深圳市海柔创新科技有限公司,海柔创新科技,会计实习生,3500,7000,1,2,2,不限,2,2024-01-01 00:00,"['电子商务', '互联网']","['互联网', '人工智能']",[],1.深入了解区域项目，搭建项目概算模板；2.负责中小项目的项目概算工作；3.负责历史概算项目...,"['餐饮补贴', '专业培训', '弹性工作']",广东省深圳市宝安区西乡街道南昌社区固戍二路99号(南昌第二工业区东区)
1,1631112859897430016,深圳市海柔创新科技有限公司,海柔创新科技,技术服务工程师,5000,10000,1,2,2,不限,5,2023-12-31 00:00,"['电子商务', '互联网']","['互联网', '人工智能']",[],工作职责1、负责客户项目交付后的运维工作，协助售后服务工程师支持客户现场业务操作，现场异常处...,"['餐饮补贴', '专业培训', '弹性工作']",广东省深圳市宝安区西乡街道南昌社区固戍二路99号(南昌第二工业区东区)
2,1629824814967554048,广州太普软件科技有限公司,太普软件,大数据分析师（BI）,4500,7000,1,2,3,不限,1,2023-12-31 00:00,"['互联网', '软件']","['互联网', '软件']",[],岗位要求：(1)大学本科及以上学历，计算机、数学相关专业；(2)熟悉SQL、SAS、SPSS...,"['餐饮补贴', '专业培训', '弹性工作']",广州市天河区高普路1023号1楼101室自编23号


In [4]:
for job_wage_kind in frame_data['job_wage_kind'].unique():
    print(job_wage_kind, frame_data[frame_data['job_wage_kind'] == job_wage_kind].iloc[0, 0])

job_wage_kind_json = { 0: '年', 1: '月', 2: '日'}

1 1631112859985510400
2 1495652137043099648
0 1374177686322286592


In [5]:
for require_kind in frame_data['require_kind'].unique():
    print(require_kind, frame_data[frame_data['require_kind'] == require_kind].iloc[0, 0])

require_kind_json = { 0: '实习', 1: None, 2: '全职'}

2 1631112859985510400
0 1613439889204969472


In [6]:
for require_edu in frame_data['require_edu'].unique():
    print(require_edu, frame_data[frame_data['require_edu'] == require_edu].iloc[0, 0])

require_edu_json = { 0: '不限', 1: '技工', 2: '大专', 3: '本科', 4: '硕士', 5: '博士'}

2 1631112859985510400
3 1629824814967554048
0 1613431494808829952
5 1554023803397472256
1 1541254101239726080
4 1515899759372664832


In [7]:
for company_addr in frame_data['company_addr'].unique()[:5]:
    addr = cpca.transform([company_addr])
    print(company_addr[:6], addr['省'][0], addr['市'][0])

广东省深圳市 广东省 深圳市
广州市天河区 广东省 广州市
广州大学城( 广东省 广州市
湖北省武汉市 湖北省 武汉市
四川省宜宾市 四川省 宜宾市


In [8]:
# for company_addr_code in frame_data['company_addr_code'].unique():
#     print(company_addr_code, frame_data[frame_data['company_addr_code'] == company_addr_code].iloc[0, 0])

In [9]:
save_frame = {"序号": [], '招聘信息 ID': [], '企业名称': [], '企业类型': [], '招聘岗位': [], '薪资范围': [], '招募类型': [], 
              '期望学历': [], '工作年数': [], '招募人数': [], "岗位类型": [], "技能需求": [], "公司地址": [], "工作福利": []}

TEST, RECORD, count = True, 5, 0
size = frame_data.shape[0]

for inx in range(size):
    if TEST and inx % RECORD == 0: continue
    item = frame_data.iloc[inx, :]
    count += 1
    save_frame['序号'].append(count)
    save_frame['招聘信息 ID'].append('\t' + str(item['job_id']))
    save_frame['企业名称'].append(item['company_full_name'])
    save_frame['企业类型'].append(item['company_ind'])
    save_frame['招聘岗位'].append(item['position_name'])
    wage_range = [item['job_min_wage'], item['job_max_wage']]
    save_frame['薪资范围'].append([wage_range[0] // 12, wage_range[1] // 12] if item['job_wage_kind'] == 0 \
        else ([wage_range[0] * 30, wage_range[1] * 30] if item['job_wage_kind'] == 2 else wage_range))
    save_frame['招募类型'].append(require_kind_json[item['require_kind']])
    save_frame['期望学历'].append(require_edu_json[item['require_edu']])
    save_frame['工作年数'].append('不限' if '不限' in item['require_exp'] else (
        item['require_exp'] if '年' in item['require_exp'] else item['require_exp'] + '年'))
    save_frame['招募人数'].append(item['num_people'])
    save_frame['岗位类型'].append(item['position_keys'])
    save_frame['技能需求'].append(item['skill_keys'])
    addr1, addr2 = cpca.transform([item['company_addr']]), cpca.transform([item['company_full_name']])
    province = addr1['省'][0] if addr1['省'][0] else (addr2['省'][0] if addr2['省'][0] else '')
    city = addr1['市'][0] if addr1['市'][0] else (addr2['市'][0] if addr2['市'][0] else '')
    save_frame['公司地址'].append((province + city).replace("市县", '市'))
    save_frame['工作福利'].append(item['job_welfare'])

print(size, len(save_frame['序号']))
# for key, value in save_frame.items():
#     print(key, len(value))

save_frame = pd.DataFrame(save_frame)
save_frame.to_csv(save_csv_path, index=False, encoding="GBK", errors='ignore')

1568 1254
